In [1]:
from Diagnosis import df
import pandas as pd


Loading processed data from pickle file...


In [2]:
df.head()

,Date,Product,Channel,EMR#,Consultation #,Mobile #,Member #,Type,Status,Patient Gender,...,Doctor,Chief Complaint,Diagnosis,Advice,REFERRED,Lab Tests,Medicine,Clinic Name,Prescrption json,ICD Code
0,2020-05-15,M-Swasth Doctor - FREE,CashPor,1,1,7571029662,21:01:84:01:25,CALL,DONE,M,...,SATISH,Other,.,referred to higher centre,NO,NaN,NO,NaN,"{""pres"":[]}",.
1,2021-06-10,M-Swasth Doctor - FREE,CashPor,1,47576,7571029662,21:01:84:01:25,CALL,DONE,M,...,KADAMBARI,Other,Dysmenorrhea and menorrhagia,Advised not miss a single tablet of Ovral l\nD...,NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Tab Ovral L"",""m"":"""",""a"":"""",...",NONE
2,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48043,7571029662,21:01:84:01:25,CALL,DONE,M,...,NIKHIL,Other,C/o Weakness with loss of appetite,"Eat healthy food , green vegetables and fruits...",NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Multiprex Cap"",""m"":""1"",""a"":...",NONE
3,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48603,7571029662,21:01:84:01:25,CALL,DONE,M,...,ravindra,Other,pain,Take nutritious diet\nDo Excercise regularly\n...,NO,CBC,YES,NaN,"{""pres"":[{""mName"":""Paracetamol 500mg Tab"",""m"":...",R50
4,2021-06-16,M-Swasth Doctor - FREE,CashPor,1,52282,7571029662,21:01:84:01:25,CALL,DONE,M,...,Harinder Pal,Other Pain,Gall Stones,Referred to nearby hospital for surgical consu...,YES,Ultrasound,NO,NaN,"{""pres"":[]}",K87


In [3]:
import os
import pandas as pd

# Define the path to the pickle file
pickle_file_path = 'clinic_cleaned.pkl'

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    df = pd.read_pickle(pickle_file_path)
    print(f"Pickle file loaded from: {pickle_file_path}")
else:


    column_to_clean = 'Diagnosis'

    df[column_to_clean] = df[column_to_clean].str.replace('?', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('c/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('K/C/O HTN', '', regex=False)
    df[column_to_clean] = df[column_to_clean].replace(r'\\n', ',', regex=True).replace(r'\n', ',', regex=True)
    df[column_to_clean] = df[column_to_clean].str.replace('u/E', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('U/E', '', regex=False)

    # Deleting column
    column_to_delete = 'Diagonis final'
    if column_to_delete in df.columns:
        df.drop([column_to_delete], axis=1, inplace=True)
    else:
        print(f"Column '{column_to_delete}' not found in the DataFrame.")

    df[column_to_clean] = df[column_to_clean].str.lower()

    # Save the cleaned DataFrame to a pickle file
    df.to_pickle(pickle_file_path)
    print(f"Pickle file saved to: {pickle_file_path}")

# Display the updated DataFrame


Pickle file loaded from: clinic_cleaned.pkl


In [4]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df = df[df['Date'].dt.year >= 2024]

In [5]:
df['Diagnosis'] = df.apply(
    lambda row: row['Chief Complaint'] if pd.isnull(row['Diagnosis']) and isinstance(row['Chief Complaint'], str) and 
    row['Chief Complaint'].lower() != 'other' and row['Chief Complaint'].lower() != 'other pain' else row['Diagnosis'], 
    axis=1
)


In [6]:
df["Diagnosis"].dropna(inplace=True)


In [7]:
df = df.sort_values(by='Date', ascending=True)


In [8]:
df.columns

Index(['Date', 'Product', 'Channel', 'EMR#', 'Consultation #', 'Mobile #',
       'Member #', 'Type', 'Status', 'Patient Gender', 'FEMALE DOC ONLY',
       'Doctor', 'Chief Complaint', 'Diagnosis', 'Advice', 'REFERRED',
       'Lab Tests', 'Medicine', 'Clinic Name', 'Prescrption json', 'ICD Code'],
      dtype='object')

In [9]:
df['Diagnosis'].value_counts()

Diagnosis
urti                                                                          35828
weakness                                                                      31393
cough                                                                         20405
weakness                                                                      20372
knee pain                                                                     17860
                                                                              ...  
heel pain , shoulder pain                                                         1
injury to shoulder since 10yrs                                                    1
anoraxia /weakness                                                                1
low backache with tingling sensation legs - since 1 month,htn                     1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism         1
Name: count, Length: 318349, dtype: int64

In [10]:
df['Diagnosis'] = df['Diagnosis'].str.strip().str.lower()
df= df[df['Diagnosis'] != '']

In [11]:
df['Diagnosis'].value_counts()

Diagnosis
weakness                                                                     52753
urti                                                                         43466
cough                                                                        37441
knee pain                                                                    25604
headache                                                                     21912
                                                                             ...  
fever , cough, cold since 1 week                                                 1
typhoid,gastritis,weakness                                                       1
numbness rt thigh                                                                1
fever ,body pain ,back pain                                                      1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism        1
Name: count, Length: 297904, dtype: int64

In [12]:
import pandas as pd
import re

# Create a regex pattern to split by multiple delimiters
delimiters = [',', '/', '&', ' and ', ' , ', 'with']
pattern = '|'.join(map(re.escape, delimiters))

# Function to split and explode the Diagnosis column


def split_and_explode(df, column, pattern):
    # Split the column by the pattern and create a new DataFrame with exploded values
    s = df[column].str.split(pattern, expand=True).stack()
    s = s.str.strip()  # Remove leading/trailing whitespaces
    # Adjust index to match the original DataFrame
    s.index = s.index.droplevel(-1)
    return df.drop(columns=[column]).join(s.rename(column))


# Apply the function to the DataFrame
result_data = split_and_explode(df, 'Diagnosis', pattern)

# Remove duplicates and reset the index
result_data = result_data.drop_duplicates().reset_index(drop=True)



In [13]:
result_data['Diagnosis'].value_counts()

Diagnosis
weakness                                                                     246682
cough                                                                        152530
gastritis                                                                     89980
headache                                                                      78070
fever                                                                         75091
                                                                              ...  
itchy lesion over thigh                                                           1
cold - allergic in nature                                                         1
eyeache since 2 days                                                              1
rhinorrhea - since 2 days                                                         1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism         1
Name: count, Length: 112643, dtype: int64

In [14]:
value_counts = result_data['Diagnosis'].value_counts()

# Step 2: Filter out values with counts less than 10
to_keep = value_counts[value_counts >= 100].index

# Step 3: Update the DataFrame to only keep the rows with the filtered values
result_data = result_data[result_data['Diagnosis'].isin(to_keep)]
result_data['Diagnosis'].value_counts()

Diagnosis
weakness                      246682
cough                         152530
gastritis                      89980
headache                       78070
fever                          75091
                               ...  
tingling sensation in arms       100
miliaria                         100
dmt2                             100
cough x5d                        100
wekness                          100
Name: count, Length: 896, dtype: int64

In [15]:
reverse_mapping = {
    'weakness': ['weak ness','asthenia','weakness', 'generalized weakness', 'generalized body weakness', 'generalised weakness',  'weaknes', 'weekness', 'weakness.', 'bodyache weakness', 'bodypain weakness', 'headache weakness', 'fever weakness', 'weakness since 8days',"bodyache,weakness","1 weakness"],
    'cough': ['cough', 'dry cough', 'cough.', 'coughing', 'cough cold', 'cough x3d', 'productive cough', 'non productive', 'fever cough', 'fever cough cold', 'cough cold since 2days', 'wet cough', 'acute cough', 'cough.', 'flu like symptoms-cough', 'fever cold cough'],
    'gastritis': ['gastritis', 'acute gastritis', 'gas', 'hyperacidity', 'gerd', 'indigestion', 'acidity', 'acidity.', 'acid reflux'],
    'headache': ['headache', 'headache.', 'head ache', 'migraine','headace','headche'],
    'fever': ['fever', 'viral fever', 'seasonal fever', 'fever x3d', 'mild fever.', 'pyrexia'],
    'urti': ['urti', 'upper respiratory tract infection', 'urti features', 'uri', 'upper respiratory tract infection c weakness', 'upper respiratory tract infection c weakness c anorexia'],
    'knee pain': ['knee','l knee','knee ache', 'knee pain', 'knee pain b', 'kneepain', 'knee pain [ weakness ]', 'knee pain', 'knee joint pain', 'knee-pain', 'knee arthralgia'],
    'cold': ['cold', 'common cold', 'common cold weakness', 'cold cough', ' cold', 'cold and cough'],
    'body pain': ['pain in body', 'generalised body pain', 'bodyache since 1month', 'bodyache', 'body ache', 'bodyaches', 'generalized body ache', 'body pain', 'body pain.', 'body pain', 'body pain and. weakness', 'generalised body pain', 'generalized body pain', 'body pains', 'body pain weakness', 'generalized body ache','generalised body pain'],
    'back pain': ['backache','back ache', 'back aches', 'back ache.', 'backcahe', 'back pain', 'backpain', 'pain back', 'pain in back', 'back ache.', 'backcahe', 'back pain', 'backpain', 'pain back', 'pain in back', ],
    'low back pain' : ['lpb', 'low back pain', 'low backache', 'low back ache', 'lba', 'lower back pain', 'lower back ache', 'pain lower back', 'lumbago'],
    'leg pain': ['leg pain', 'pain in legs', 'l leg pain', 'left leg pain', 'legpain', 'right leg pain', 'pain in lower limb', 'leg.', 'leg', 'pain in leg'],
    'myalgia': ['myalgia', 'myalgia [ weakness ]', 'myalgia b'],
    'ringworm': ['ringworm', 'ring worm', 'ringworm infection', 'ring worm infection', 'tinea', 'tinea corporis', 'tinea cruris'],
    'anorexia': ['anorexia', 'loss of appetite', 'anorexia'],
    'leucorrhoea': ['leucorrhoea', 'leucorrhea', 'leukorrhea'],
    'hypertension': ['hypertension', 'htn', 'high bp'],
    'hypotension': ['hypotension', 'low bp'],
    'toothache': ['toothache', 'tooth ache', 'tooth pain', 'dental caries'],
    'chest pain': ['chest pain'],
    'itching': ['itching', 'pruritus', 'pruritis', 'itching over body', 'itching all over body', 'itchiness', 'skin itching'],
    'malaise': ['malaise', 'generalized malaise'],
    'shoulder pain': ['shoulder pain', 'pain shoulder', 'right shoulder pain', 'left shoulder pain'],
    'joint pain': ['joint pain', 'arthralgia', 'arthalgia', 'joint pain', 'arthralgia knee', 'polyarthralgia'],
    'fungal infection': ['fungal infection'],
    'arthritis': ['arthritis', 'osteoarthritis'],
    'rhinitis': ['allergic rhinitis', 'rhinitis'],
    'gastrointestinal symptoms': ['gastrointestinal symptoms', 'gastroenteritis'],
    'diabetes': ['diabetes', 'diabetes mellitus', 'dm', 't2dm'],
    'skin problems': ['skin problems', 'skin problem', 'skin infection', 'skin allergy', 'eczema', 'dermatitis', 'contact dermatitis', 'dandruff', 'xeroderma'],
    'anemia': ['anaemia', 'anemia', 'anemia'],
    'dizziness': ['dizziness', 'giddiness'],
    'vomiting': ['vomiting', 'vomitting','vomit','pukish'],
    'nausea': ['nausea'],
    'swelling': ['swelling', 'oedema', 'edema'],
    'dehydration': ['dehydration'],
    'hair loss': ['hairfall', 'hair fall'],
    'urinary tract infection': ['uti', 'urinary tract infection'],
    'body pain': ['body pain', 'generalised body pain', 'generalized body pain'],
    'visual acuity': ['visual acuity', 'refractive error'],
    'pain': ['pain', 'other pain','generalised pain'],
    'abdominal pain': ['pain abdomen', 'pain in abdomen', 'abdominal pain',],
    'sore throat': ['sore throat', 'pharyngitis'],
    'ear pain': ['ear pain', 'earache'],
    'neck pain': ['neck pain'],
    'allergy': ['allergy', 'skin allergy', 'itching over body', 'itching all over body'],
    'nutritional deficiency': ['nutritional deficiency', 'vitamin deficiency', 'nutritional def', 'vit deficiency', 'calcium deficiency'],
    'hypothyroidism': ['hypothyroidism'],
    'diabetes': ['diabetes', 'diabetes mellitus'],
    'spondylosis': ['spondylosis', 'cervical spondylosis'],
    'loose motion' : ['loose motions','loose motion']
    # Add more mappings as needed
}

In [16]:
def find_diagnosis(chief_complaint):
    for category, symptoms in reverse_mapping.items():
        for symptom in symptoms:
            if symptom in chief_complaint.lower():
                return category
    return chief_complaint  # return original if no match found

# Apply the mapping to the 'Diagnosis' column
result_data['Diagnosis'] = result_data['Diagnosis'].apply(find_diagnosis)
result_data['Diagnosis'].value_counts()

Diagnosis
weakness                      276204
cough                         188203
gastritis                     122947
back pain                     116575
headache                       84687
                               ...  
high b.p.                        100
wekness                          100
filariasis                       100
dmt2                             100
tingling sensation in arms       100
Name: count, Length: 415, dtype: int64

In [17]:
import numpy as np

In [18]:
def remove_invalid_diagnoses(df, column_name='Diagnosis'):
    """
    Remove blank entries and single-letter entries from the specified column.
    
    Args:
    df (pandas.DataFrame): The input DataFrame
    column_name (str): The name of the column to process (default is 'Diagnosis')
    
    Returns:
    pandas.DataFrame: The DataFrame with invalid entries removed
    """
    # Replace empty strings, whitespace-only strings, and single-letter entries with NaN
    df[column_name] = df[column_name].replace(r'^\s*$', np.nan, regex=True)
    df[column_name] = df[column_name].replace(r'^\s*.\s*$', np.nan, regex=True)
    df[column_name] = df[column_name].replace(r'^\d+.*$', np.nan, regex=True)

    
    # Remove rows where the specified column is NaN
    df = df.dropna(subset=[column_name])
    
    # Strip leading and trailing whitespace from the remaining entries
    df[column_name] = df[column_name].str.strip()
    


In [19]:
remove_invalid_diagnoses(result_data)

C:\Users\siidh\AppData\Local\Temp\ipykernel_13712\794629077.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.strip()


In [20]:
from rapidfuzz import process, fuzz


In [21]:
def group_similar_diagnoses(df, threshold=60):
    # Get unique diagnoses
    unique_diagnoses = df['Diagnosis'].unique()
    
    # Create a mapping of similar diagnoses
    mapping = {}
    for diagnosis in unique_diagnoses:
        if diagnosis not in mapping:
            mapping[diagnosis] = diagnosis
            
            # Find similar diagnoses
            matches = process.extract(diagnosis, unique_diagnoses, scorer=fuzz.ratio, limit=None)
            for match in matches:
                if len(match) >= 2:  # Ensure we have at least two elements
                    match_str, score = match[0], match[1]
                    if score >= threshold and match_str != diagnosis:
                        mapping[match_str] = diagnosis
    
    # Apply the mapping to the dataframe
    df['Diagnosis'] = df['Diagnosis'].map(mapping)
    
    return df

In [22]:
result_data = group_similar_diagnoses(result_data)

result_data["Diagnosis"].value_counts()

Diagnosis
weakness                                                 276576
cough                                                    188203
joint pain                                               149462
gastritis                                                122947
back pain                                                116575
                                                          ...  
pt is hypertensive but did not started any medication       108
gout                                                        104
cholelithiasis                                              102
pregnant                                                    101
high b.p.                                                   100
Name: count, Length: 181, dtype: int64

In [23]:
diagnosis_map = {
    # General symptoms
    'general_weakness': ['weakness', 'malaise', 'fatigue', 'generalized fatigue', 'tiredness', 'generalized', 'wekaness', 'lethargy', 'weakenss'],
    'low_appetite': ['low appetite', 'decrease appetite.', 'lack of appetite', 'loss of apetite', 'bad appetite', 'decreased appetite'],
    'fever': ['fever', 'puo', 'acute febrile illness', 'fuo', 'febrile illness'],


    # Pain categories
    'headache': ['headache', 'headach', 'frontal', 'headcahe'],
    'back_pain': ['back pain', 'low back pain', 'lbp', 'back ache', 'low back'],
    'joint_pain': ['joint pain', 'arthritis', 'knee pain', 'shoulder pain', 'arthropathy', 'shoulder'],
    'body_pain': ['body pain', 'bodyache', 'myalgia', 'generalized body ache', 'bodyaches', 'gen body ache', 'body aches','body ache'],
    'chest_pain': ['chest pain'],
    'abdominal_pain': ['gastritis', 'stomachache', 'abdominal ache', 'abdomen ache'],
    'hand_pain': ['hand', 'l hand', 'hand\'s', 'hands'],
    'foot_pain': ['feet', 'plantar fasciitis', 'plantar fascitis'],
    'neck_pain': ['neck', 'cervical disc disorder', 'cervical spondylitis'],
    'neuralgia': ['neuralgia'],

    # Respiratory
    'upper_respiratory': ['cough', 'urti', 'cold', 'sore throat', 'rhinitis', 'runny nose', 'sinusitis', 'acute sinusitis', 'upper respiratory tract (ent) symptoms', 'sneezing', 'upper respiratory infection'],
    'lower_respiratory': ['lrti', 'shortness of breath', 'breathlessness', 'sob', 'asthma', 'dyspnea', 'breathing difficulty', 'copd', 'dyspnoea', 'bronchitis'],
    'respiratory_symptoms': ['sputum', 'expectoration'],
    'seasonal_flu': ['seasonal flu', 'viral illness'],
    'allergic_airway': ['allergic airway disease'],

    # ENT
    'ear_infections': ['otitis media', 'ear infection', 'csom'],
    'ear_symptoms': ['ear discharge', 'ear pain', 'ear ache', 'tinnitus', 'hearing loss'],
    'nose_symptoms': ['epistaxis', 'rhinorrhoea'],
    'throat_conditions': ['tonsillitis'],

    # Gastrointestinal
    'digestive_issues': ['anorexia', 'constipation', 'flatulence', 'dyspepsia', 'heartburn'],
    'diarrheal_diseases': ['loose motion', 'diarrhoea', 'loose stools', 'loose stool', 'dysentry'],
    'nausea_vomiting': ['nausea', 'vomiting'],
    'anal_conditions': ['piles', 'hemorrhoids', 'haemorrhoids'],
    'ibs': ['ibs', 'irritable bowel syndrome'],
    'fatty_liver': ['fatty liver'],

    # Skin conditions
    'fungal_skin': ['ringworm', 'fungal infection', 'dermatophytosis', 't.corporis', 'tenia corporis', 't. corporis', 'tenia inf', 'mycosis', 'ptyriasis versicolor', 'onychomycosis'],
    'parasitic_skin': ['scabies', 'scabie'],
    'allergic_skin': ['itching', 'allergy', 'itchy skin', 'utricaria'],
    'acne': ['acne', 'pimples', 'pimple'],
    'other_skin': ['skin problems', 'rash', 'melasma', 'boils', 'rashes', 'dermatities', 'dry skin', 'boil', 'cracked heels', 'skin disease', 'cracked heel', 'hyperpigmentation', 'psoriasis', 'vitiligo'],

    # Gynecological
    'menstrual_disorders': ['dysmenorrhea', 'menorrhagia', 'dysmenorrhoea', 'oligomenorrhoea', 'irregular menstrual cycle', 'oligomenorrhea', 'irregular menses', 'polymenorrhoea', 'irregular menstruation', 'polymenorrhea', 'amenorrhea', 'dub', 'irregular periods', 'amenorrhoea', 'irregular period'],
    'vaginal_discharge': ['leucorrhoea', 'white discharge', 'leuchorrhea', 'leukorrhoea', 'leucorrohea', 'pv-discharge'],
    'pelvic_disorders': ['pid', 'pcod'],
    'vaginitis': ['vaginitis', 'bacterial vaginosis'],
    'pregnancy_related': ['lactating mother', 'breastfeeding+', 'p2l2'],
    'infertility': ['infertility'],

    # Cardiovascular
    'hypertension': ['hypertension', 'raised bp', 'high blood pressure documented'],
    'hypotension': ['hypotension', 'low b.p.', 'low blood pressure documented'],
    'heart_symptoms': ['palpitations', 'heart_symptoms', 'palpitation'],
    'varicose_veins': ['varicose veins', 'varicose vein'],
    'ihd': ['ihd', 'cad'],

    # Metabolic and Nutritional
    'diabetes': ['diabetes', 'dm', 't2dm'],
    'thyroid_disorders': ['hypothyroidism', 'thyroid', 'thyroid disease'],
    'nutritional_deficiencies': ['nutritional deficiency', 'anemia', 'hypovitaminosis', 'malnutrition', 'pica'],
    'obesity': ['obesity'],

    # Musculoskeletal
    'arthritis': ['arthritis', 'oa', 'oa knee', 'athralgia'],
    'back_disorders': ['spondylosis', 'spondylitis'],
    'bone_disorders': ['osteoporosis', 'bone_disorders'],
    'sprain_and_strain': ['sprain', 'strain', 'mild sprain'],
    'frozen_shoulder': ['frozen shoulder'],

    # Neurological
    'headache_and_migraine': ['headache'],
    'dizziness_vertigo': ['dizziness', 'vertigo'],
    'neuropathy': ['peripheral neuropathy', 'neuropathy'],
    'numbness_tingling': ['numbness', 'tingling sensation', 'tingling', 'paresthesia ll', 'paresthesia'],

    # Eye problems
    'eye_infections': ['conjunctivitis', 'eye infection', 'conjuctivitis', 'eye flu'],
    'eye_symptoms': ['visual acuity', 'eye problem', 'eye problems', 'eye discharge', 'eye redness', 'eye irritation', 'diminished vision', 'blurring of vision', 'discharge from eye', 'blurred vision', 'epiphora', 'decreased vision', 'low vision', 'diminished visual acquity', 'watery eye', 'watery eyes', 'difficulty in vision', 'blurring of vision)', 'watering from eyes'],
    'refractive_errors': ['myopia', 'hypermetropia'],
    'cataract': ['cataract'],
    'dry_eye': ['dry eye', 'dry eyes'],

    # Oral conditions
    'oral_issues': ['mouth ulcer', 'oral ulcer', 'oral ulcers', 'gingivitis', 'aphthous ulcer', 'gum infection'],

    # Infections
    'viral_infections': ['viral infection', 'mumps'],
    'parasitic_infections': ['worm infestation', 'worms', 'ascariasis', 'taeniasis', 'worm'],
    'typhoid': ['typhoid'],

    # Urinary system
    'urinary_tract_infections': ['urinary tract infection'],
    'urinary_symptoms': ['burning micturation'],
    'renal_issues': ['renal stone', 'renal calculi', 'renal stones'],

    # Mental health
    'anxiety': ['anxiety'],
    'sleep_disorders': ['insomnia'],

    # Other specific conditions
    'hair_issues': ['hair loss', 'alopecia'],
    'wounds_and_injuries': ['wound', 'trauma', 'injury', 'o injury', 'o fall'],
    'dehydration': ['dehydration'],
    'chronic_diseases': ['chronic diseases'],
    'hydrocele': ['hydrocele'],
    'spasm': ['spasm'],

}

In [24]:
result_data.shape

(1852745, 21)

In [25]:
def find_diagnosi(chief_complaint):
    # Convert to string and handle NaN values
    if pd.isna(chief_complaint):
        return chief_complaint
    
    chief_complaint_str = str(chief_complaint).lower()
    
    for category, symptoms in diagnosis_map.items():
        for symptom in symptoms:
            if symptom.lower() in chief_complaint_str:
                return category
    return chief_complaint  # return original if no match found



# Apply the mapping to the 'Diagnosis' column
result_data['Diagnosis'] = result_data['Diagnosis'].apply(find_diagnosi)
result_data['Diagnosis'].value_counts()

Diagnosis
upper_respiratory                                        351580
general_weakness                                         291881
joint_pain                                               158050
back_pain                                                136426
abdominal_pain                                           132683
                                                          ...  
pt is hypertensive but did not started any medication       108
gout                                                        104
cholelithiasis                                              102
pregnant                                                    101
high b.p.                                                   100
Name: count, Length: 99, dtype: int64

In [26]:
result_data.dropna(subset=['Clinic Name'], inplace=True)


In [67]:

result_data_final=result_data[["Channel","Clinic Name","Diagnosis","Date"]]
top_20_diagnoses = result_data_final['Diagnosis'].value_counts().nlargest(20).index
result_data_final = result_data_final[result_data_final['Diagnosis'].isin(top_20_diagnoses)]


In [68]:
result_data_final

,Channel,Clinic Name,Diagnosis,Date
34,Satya Microcapital,ECS018 Sarawa,general_weakness,2024-01-01
35,Satya Microcapital,ECS018 Sarawa,back_pain,2024-01-01
38,Satya Microcapital,ECS008 chandpuri khadar haridwar,leg pain,2024-01-01
40,Satya Microcapital,ECS008 chandpuri khadar haridwar,general_weakness,2024-01-01
41,Satya Microcapital,ECS015 Phulgarh,general_weakness,2024-01-01
...,...,...,...,...
2156696,Namra Finance,ECN160 CHAKIA,upper_respiratory,2024-05-22
2156697,Utkarsh SFB,ECU030 Jungle Subhanali,parasitic_skin,2024-05-22
2156698,Muthoot,ECMT133 RAMNAGAR MML WEST CHAMPARAN,leg pain,2024-05-22
2156700,Svatantra,ECSV0356 Jamui Jamui,joint_pain,2024-05-22


In [30]:
# import pandas as pd

# def generate_hierarchical_report(df):
#     # Ensure 'Date' column is in datetime format
#     df['Date'] = pd.to_datetime(df['Date'])
    
#     # Extract month from the date
#     df['Month'] = df['Date'].dt.strftime('%b')  # Short month name (e.g., 'Jan')

#     # Group by Month, Clinic Name, and Diagnosis, and count occurrences
#     grouped = df.groupby(['Month', 'Clinic Name', 'Diagnosis']).size().reset_index(name='Count')

#     # Sort the dataframe
#     grouped = grouped.sort_values(['Month', 'Clinic Name', 'Count'], ascending=[True, True, False])

#     # Generate the report
#     report = ""
#     for month, month_data in grouped.groupby('Month'):
#         month_total = month_data['Count'].sum()
#         report += f"{month}: {month_total}\n"
        
#         for clinic, clinic_data in month_data.groupby('Clinic Name'):
#             clinic_total = clinic_data['Count'].sum()
#             report += f"  {clinic}: {clinic_total}\n"
            
#             for _, row in clinic_data.iterrows():
#                 report += f"    {row['Diagnosis']}: {row['Count']}\n"
        
#         report += "\n"

#     return report

# # Assuming your dataframe is named 'result_data'
# report = generate_hierarchical_report(result_data)

# # Print the report
# print(report)

# # Optionally, save the report to a text file
# with open('diagnosis_report.txt', 'w') as f:
#     f.write(report)

In [31]:
# import pandas as pd
# import numpy as np
# from rapidfuzz import process, fuzz

# def create_canonical_forms(df, column_name, threshold=80):
#     """
#     Create a dictionary of canonical forms based on the most common diagnoses using RapidFuzz.
    
#     Args:
#     df (pandas.DataFrame): The input DataFrame
#     column_name (str): The name of the column containing diagnoses
#     threshold (int): The similarity threshold for fuzzy matching
    
#     Returns:
#     dict: A dictionary mapping similar diagnoses to their canonical forms
#     """
#     # Split multi-diagnoses and create a flat list, handling NaN values
#     all_diagnoses = [
#         diagnosis.strip().lower() 
#         for diagnoses in df[column_name].dropna().astype(str).str.split(',') 
#         for diagnosis in diagnoses
#     ]
    
#     # Get unique diagnoses sorted by frequency
#     unique_diagnoses = pd.Series(all_diagnoses).value_counts().index.tolist()
    
#     canonical_forms = {}
#     mapped_diagnoses = set()
    
#     for diagnosis in unique_diagnoses:
#         if diagnosis not in mapped_diagnoses:
#             try:
#                 similar_diagnoses = process.extract(diagnosis, unique_diagnoses, limit=None, scorer=fuzz.ratio)
#                 for match in similar_diagnoses:
#                     if len(match) >= 2:
#                         similar, score = match[0], match[1]
#                         if score >= threshold and similar not in mapped_diagnoses:
#                             canonical_forms[similar] = diagnosis
#                             mapped_diagnoses.add(similar)
#                     else:
#                         print(f"Unexpected match format for diagnosis '{diagnosis}': {match}")
#             except Exception as e:
#                 print(f"Error processing diagnosis '{diagnosis}': {str(e)}")
    
#     return canonical_forms

# def map_diagnosis(diagnosis, canonical_forms):
#     """Map a single diagnosis to its canonical form."""
#     if pd.isna(diagnosis):
#         return np.nan
#     return canonical_forms.get(str(diagnosis).strip().lower(), diagnosis)

# def process_diagnoses(df, column_name):
#     """Process the diagnoses column and map to canonical forms."""
#     # Create canonical forms
#     canonical_forms = create_canonical_forms(df, column_name)
    
#     # Function to process a single cell (which may contain multiple diagnoses)
#     def process_cell(cell):
#         if pd.isna(cell):
#             return np.nan
#         diagnoses = [d.strip() for d in str(cell).split(',')]
#         mapped_diagnoses = [map_diagnosis(d, canonical_forms) for d in diagnoses]
#         return ', '.join(filter(lambda x: not pd.isna(x), mapped_diagnoses))
    
#     # Apply the processing to the entire column
#     df[column_name] = df[column_name].apply(process_cell)
    
#     return df

# # Assuming you have a DataFrame named 'df' with a 'Diagnosis' column
# # df = pd.read_csv('your_data.csv')  # Uncomment this line if you need to load the data



In [32]:
# # Assuming you have a DataFrame named 'df' with a 'Diagnosis' column
# # df = pd.read_csv('your_data.csv')  # Uncomment this line if you need to load the data

# # Process the diagnoses
# result_data = process_diagnoses(result_data, 'Diagnosis')

# # Display the updated value counts
# print(result_data['Diagnosis'].value_counts())



In [33]:
# result_data['Diagnosis'] = result_data['Diagnosis'].apply(map_diagnosis)


In [34]:
# result_data['Diagnosis'].value_counts().to_csv("1.csv")


ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD